In [ ]:
import numpy as np
import pandas as pd

from OptimalHedging.GBM import GBMSimulator  # conforme você pediu

# -----------------------------
# 0) Parâmetros
# -----------------------------
M     = 1000
K     = 150.0
S0    = 100.0
T     = 1.0
t0    = 0.0
mu    = 0.05
sigma = 0.2

gamma = 2.0
a     = 2.0
k     = 2.0
beta  = 0.95

np.random.seed(123)

# -----------------------------
# 1) Instancia + simula S e H
# -----------------------------
sim = GBMSimulator(
    M=M, K=K, S0=S0, T=T, t0=t0, mu=mu, sigma=sigma
)

sim.simulate_S()
sim.simulate_H()

# -----------------------------
# 2) Riscos: definição de kwargs
# -----------------------------
risk_specs = {
    "ele" : {"a": a},
    "elw" : {"k": k},
    "entl": {"gamma": gamma},
    "ente": {"gamma": gamma, "a": a},
    "entw": {"gamma": gamma, "k": k},
    "esl" : {"beta": beta},
}

# nomes de colunas no estilo da sua tabela
col_name = {
    "ele":  "EL-exp",
    "elw":  "EL-W",
    "entl": "Ent-L",
    "ente": "Ent-exp",
    "entw": "Ent-W",
    "esl":  "ES",
}

# Ordem de avaliação nas colunas (igual ao print do exemplo)
eval_order = ["ele", "elw", "entl", "ente", "entw", "esl"]

# -----------------------------
# 3) Loop: otimiza e avalia
# -----------------------------
rows = []
hedges = {}

for opt_risk in risk_specs.keys():
    h_opt, hist = sim.optimize_hedge(
        risk_type=opt_risk,
        risk_kwargs=risk_specs[opt_risk],
        max_iter=20,
        tol=1e-4,
        alpha=1e-2,
        verbose=True
    )
    hedges[opt_risk] = h_opt

    # Terminal P&L do hedge ótimo para esse risco
    L  = sim.forward_PL(h_opt)
    LT = L[:, -1]

    # Avalia LT sob todas as medidas
    row = {"Model": "GBM", "Optimized risk": col_name[opt_risk]}
    for ev in eval_order:
        row[col_name[ev]] = sim.risk_function(LT, ev, **risk_specs[ev])
    rows.append(row)

df = pd.DataFrame(rows, columns=["Model", "Optimized risk"] + [col_name[r] for r in eval_order])

# -----------------------------
# 4) Print no formato "paper-like"
# -----------------------------
df_rounded = df.copy()
for c in df_rounded.columns[2:]:
    df_rounded[c] = df_rounded[c].astype(float)

# você pode trocar o rounding aqui para bater com sua tabela
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
display(df_rounded)

# Opcional: tabela em LaTeX (fica muito parecido com o print)
print(df_rounded.to_latex(index=False))


In [5]:
import pandas as pd
import numpy as np
import pandas as pd

from OptimalHedging.GBM import GBMSimulator  # conforme você pediu

# -----------------------------
# 0) Parâmetros
# -----------------------------
M     = 1000
N     = 252
K     = 150.0
S0    = 100.0
T     = 1.0
t0    = 0.5
mu    = 0.05
sigma = 0.2

gamma = 2.0
a     = 2.0
k     = 2.0
beta  = 0.95
U = 5.0

np.random.seed(123)

t_idx = 0

# -----------------------------
# 1) Instancia + simula S e H
# -----------------------------
sim = GBMSimulator(
    M=M, N =N, K=K, S0=S0, T=T, t0=t0, mu=mu, sigma=sigma
)

sim.simulate_S()
sim.simulate_H()

# -----------------------------
# 2) Riscos: definição de kwargs
# -----------------------------
risk_specs = {
    "ele" : {"a": a},
    "elw" : {"k": k, "U": U},
    "entl": {"gamma": gamma, "U": U},
    "ente": {"gamma": gamma, "a": a, "U": U},
    "entw": {"gamma": gamma, "k": k, "U": U},
    "esl" : {"beta": beta},
}

# Ordem de avaliação nas colunas (igual ao print do exemplo)
eval_order = ["ele", "elw", "entl", "ente", "entw", "esl"]

# -----------------------------
# 3) Loop: otimiza e avalia
# -----------------------------
rows = []
hedges = {}

for risk_type, risk_kwargs in risk_specs.items():
    MR, info = sim.compute_MR(
        t_idx=t_idx,
        risk_type=risk_type,
        risk_kwargs=risk_kwargs,
        max_iter=20,
        tol=1e-4,
        alpha=1e-3,
        verbose=True
    )

    rows.append({
        "Model": "GBM",
        "Risk": risk_type,
        "t_idx": t_idx,
        "MR": MR,
        "rho_T": info["rho_T"],
        "rho_t": info["rho_t"],
    })

df_MR = pd.DataFrame(rows)

pd.set_option("display.float_format", lambda x: f"{x:.6f}")
display(df_MR)


iter   0 | grad_norm=5.218708e+01 | alpha=1.00e-03
    trial    | alpha=1.00e-03 | grad_norm=5.218708e+01 | reject
rollback disabled (k < 2)
iter   1 | grad_norm=5.218708e+01 | alpha=1.00e-03
    trial    | alpha=1.00e-03 | grad_norm=5.218708e+01 | reject
rollback disabled (k < 2)
iter   2 | grad_norm=5.218708e+01 | alpha=1.00e-03
    trial    | alpha=1.00e-03 | grad_norm=5.218708e+01 | reject
    rollback | anchor=curr | grad_norm=5.218708e+01 | alpha=5.00e-04
    search  0 | alpha=5.00e-04 | grad_norm=5.218708e+01 | reject
    search  1 | alpha=2.50e-04 | grad_norm=5.218708e+01 | reject
    search  2 | alpha=1.25e-04 | grad_norm=5.218708e+01 | reject
    search  3 | alpha=6.25e-05 | grad_norm=5.218708e+01 | reject
    search  4 | alpha=3.13e-05 | grad_norm=5.218708e+01 | reject
    search  5 | alpha=1.56e-05 | grad_norm=5.218708e+01 | reject
    search  6 | alpha=7.81e-06 | grad_norm=5.218708e+01 | reject
    search  7 | alpha=3.91e-06 | grad_norm=5.218708e+01 | reject
    search  8 

,Model,Risk,t_idx,MR,rho_T,rho_t
0,GBM,ele,0,-0.252000,1.000000,0.748000
1,GBM,elw,0,0.000563,1.000000,1.000563
2,GBM,entl,0,-0.127406,0.000000,-0.127406
3,GBM,ente,0,-0.072544,1.000000,0.927456
4,GBM,entw,0,0.000570,1.000000,1.000570
5,GBM,esl,0,0.148599,0.000000,0.148599


In [ ]:
# ==========================
# CONTAINERS FOR RESULTS
# ==========================

S_paths     = {}
H_paths     = {}
t_grids     = {}
Eh_curves   = {}           # Eh_curves[model][risk_type] = E[h](t)
LT_samples  = {}           # LT_samples[model][risk_type_opt] = L_T paths (under that optimized measure)
risk_rows   = []           # for the big final table


# ==========================
# MAIN LOOP
# ==========================

for model_name, factory in models.items():
    print(f"model: {model_name}")
    sim = factory()

    # 1) S and H (once per model)
    S_path = sim.simulate_S()
    sim.simulate_H()

    M_paths, N_time = S_path.shape
    t_grid = np.linspace(sim.t0, sim.T, N_time)

    S_paths[model_name] = S_path
    H_paths[model_name] = sim.H
    t_grids[model_name] = t_grid
    Eh_curves[model_name] = {}
    LT_samples[model_name] = {}

    # 2) loop over risk measures used in the optimization
    for risk_type_opt, risk_kwargs_opt in risk_specs:
        print(f"  optimized measure: {risk_type_opt}")
        h_star, history = sim.optimize_hedge(
            risk_type=risk_type_opt,
            risk_kwargs=risk_kwargs_opt,
            max_iter=max_iter,
            tol=tol,
            alpha=alpha,
            verbose=verbose,
        )

        # E[h](t) under this optimized strategy
        Eh = h_star.mean(axis=0)   # (N_time-1,)
        Eh_curves[model_name][risk_type_opt] = Eh

        # P&L under this optimized strategy
        L = sim.forward_PL(h_star, L0=0.0)
        LT = L[:, -1]
        LT_samples[model_name][risk_type_opt] = LT

        # 3) evaluate ALL risk measures on this same LT
        for risk_type_eval, risk_kwargs_eval_base in risk_specs:
            # use a copy of kwargs to avoid accidental mutation
            risk_kwargs_eval = risk_kwargs_eval_base.copy()
            rho_LT_eval = sim.risk_function(LT, risk_type_eval, **risk_kwargs_eval)
            risk_rows.append({
                "Model": model_name,
                "Optimized": risk_type_opt,
                "Evaluated": risk_type_eval,
                "rho(L_T)": rho_LT_eval,
            })


# ==========================
# BIG TABLE rho(L_T) FOR ALL COMBINATIONS
# ==========================

risk_table = pd.DataFrame(risk_rows)

# pivot so that rows = (Model, Optimized), columns = Evaluated measure
pivot_table = risk_table.pivot_table(
    index=["Model", "Optimized"],
    columns="Evaluated",
    values="rho(L_T)"
)

print(pivot_table)
